In [448]:
#USER INPUT HERE
modweight = 0.4 #change this value

#change the name of the filename
from mod_epv import mod_epv as m_epv

In [449]:
#import statements
import numpy as np
import pandas as pd
#these two imports will always be the same:
#from filename2 import matrix2 as a_epv
from r_conv_pct import conv_pct as r_conv

In [450]:
r_conv = np.matrix(r_conv)

In [451]:
#fn for ceiling division
def ceildiv(a, b):
    return -(-a // b)

#fn for grouping rows into bins of size binsize and taking the average of each bin
def binaverage(matrix, binsize):
    lst = np.array(matrix)
    output = np.zeros((len(lst), (ceildiv(len(lst[0]),binsize))))
    for i in range(0, len(lst)):
        temp = []
        for j in range(0, len(lst[0]), binsize):
            temp.append(sum(lst[i][j:j + binsize]) / binsize)
        output[i] = np.array(temp)
    return output

In [452]:
#output the average conversion percentage independent of yardline100 bin
print('conversionbydist =', binaverage(r_conv, 10))
#not necessary but cool to look at if we want to use this for something

conversionbydist = [[ 0.724459]
 [ 0.61432 ]
 [ 0.532199]
 [ 0.473218]
 [ 0.432494]
 [ 0.405153]
 [ 0.386304]
 [ 0.371073]
 [ 0.354576]]


In [453]:
#place EPVs into 10 yard bins to match conversion percentages
a_epv10 = binaverage(a_epv, 10)
m_epv10 = binaverage(m_epv, 10)

In [454]:
#print 10 yard bin averages
print(a_epv10)
print(m_epv10)

array([[ 3.99753676,  3.15235337,  2.23279822,  1.31359426,  0.3943903 ,
        -0.52481365, -1.44401761, -2.36322157, -3.28242553, -3.74010236],
       [ 3.99753676,  3.15235337,  2.23279822,  1.31359426,  0.3943903 ,
        -0.52481365, -1.44401761, -2.36322157, -3.28242553, -3.74010236],
       [ 3.99753676,  3.15235337,  2.23279822,  1.31359426,  0.3943903 ,
        -0.52481365, -1.44401761, -2.36322157, -3.28242553, -3.74010236],
       [ 3.99753676,  3.15235337,  2.23279822,  1.31359426,  0.3943903 ,
        -0.52481365, -1.44401761, -2.36322157, -3.28242553, -3.74010236],
       [ 3.99753676,  3.15235337,  2.23279822,  1.31359426,  0.3943903 ,
        -0.52481365, -1.44401761, -2.36322157, -3.28242553, -3.74010236],
       [ 3.99753676,  3.15235337,  2.23279822,  1.31359426,  0.3943903 ,
        -0.52481365, -1.44401761, -2.36322157, -3.28242553, -3.74010236],
       [ 3.99753676,  3.15235337,  2.23279822,  1.31359426,  0.3943903 ,
        -0.52481365, -1.44401761, -2.36322157

In [455]:
#create matrix compare fn with multiple use cases
def matrixcompare(m1, m2, operation):
    assert np.matrix(m1).shape == np.matrix(m2).shape, 'matrix size mismatch, cannot compare matrices of different sizes'
    lst1 = np.array(m1)
    lst2 = np.array(m2)
    output = np.zeros((len(lst1), len(lst1[0])))
    for i in range(0, len(lst1)):
        for j in range(0, len(lst1[0])):
            if operation == 'normeddiff':
                #calculate the difference between the mod and the average (how much bigger is the mod epv compared to average?)
                output[i][j] = lst2[i][j] - lst1[i][j]
            elif operation == 'combine':
                output[i][j] = (lst1[i][j] * lst2[i][j]) + lst2[i][j]
            else:
                break
        if operation == 'normeddiff':
            #normalize differences by row (normalized to be a percentage between - modweight and modweight)
            romi = -14 #these two values can be adjusted, but essentially they represent the max differences possible in 2 epvs. 
            romx = 14 #the most 2 epvs can be different is 14 or negative 14, which are the floor and ceiling of m_epv-a_epv
            output[i] = ((2 * (1 + modweight))*((np.array(output[i]) - romi) / (romx - romi)) - (1 + modweight))      
    return output

In [456]:
#testing if this concept works for r_conv
a = np.int_(np.random.rand(int(r_conv.shape[0]), int(r_conv.shape[1])) * 6)
b = m_epv10#random numbers, since this number is larger, the defense is easier
c = matrixcompare(a, b, 'normeddiff') #a is average epv, b is matchup epv. Larger EPV = easier defense = higher percentages

print('\n raw (test):\n')
print(r_conv)
print('\n adjusted (test):\n')
test = matrixcompare(c, r_conv, 'combine')
print(test)


 raw (test):

[[ 0.7037   0.69639  0.71086  0.71313  0.72533  0.73013  0.73938  0.7345
   0.75175  0.73942]
 [ 0.51983  0.60828  0.60422  0.61275  0.61886  0.64782  0.65338  0.65033
   0.64154  0.58619]
 [ 0.41503  0.53663  0.53082  0.53581  0.54087  0.57976  0.57886  0.57666
   0.55809  0.46946]
 [ 0.3685   0.4795   0.48298  0.47858  0.48557  0.52411  0.5162   0.51418
   0.49599  0.38657]
 [ 0.35945  0.43498  0.45302  0.43733  0.44713  0.47902  0.46579  0.46357
   0.44981  0.33484]
 [ 0.36708  0.40114  0.43327  0.40836  0.41974  0.44266  0.42801  0.42552
   0.41414  0.31161]
 [ 0.3706   0.37604  0.41605  0.38792  0.39759  0.41316  0.40324  0.4007
   0.38355  0.31419]
 [ 0.34922  0.35777  0.39367  0.37232  0.37486  0.38869  0.39186  0.3898
   0.35262  0.33992]
 [ 0.28213  0.34439  0.35846  0.35781  0.34574  0.3674   0.39425  0.39351
   0.31594  0.38613]]

 adjusted (test):

[[ 0.63315666  0.63736074  0.86958069  0.59286735  0.39127131  0.69181178
   0.48473623  0.48747138  0.12911866 

In [ ]:
#compare mod epv with avg epv to create a percent of average epv that represents how much different the modded epv is from the norm
epv_diff = matrixcompare(a_epv10, m_epv10, 'normeddiff') #average EPV *FIRST*. 

#multiply this with our average conversion percentages to modify conversion percentage
m_conv = matrixcompare(epv_diff, r_conv, 'combine')

In [442]:
#insert output statement here
np.savetxt("avg_conv_pct.csv", r_conv, delimiter = ',', fmt = '%10.5f')
np.savetxt("mod_conv_pct.csv", test, delimiter = ",", fmt = '%10.5f')